In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier

import timeit

In [7]:
df = pd.read_csv('selected_features.csv')
df

,verb_count,proper_noun_count,emotiveness,compound,neutral,label
0,2,2,0.200000,-0.4939,0.656,0
1,1,3,1.000000,0.3818,0.755,0
2,1,5,0.500000,0.0000,1.000,0
3,1,2,0.200000,0.0000,1.000,0
4,2,1,0.600000,0.2449,0.835,0
...,...,...,...,...,...,...
44868,1,3,0.666667,-0.3818,0.580,1
44869,1,3,0.000000,0.5267,0.764,1
44870,1,3,0.500000,0.4404,0.838,1
44871,3,2,0.250000,0.0000,1.000,1


In [8]:
# Shuffle the set
df = df.sample(frac = 1, random_state = 42)
X = df.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]
y=df.label

#Split X & y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.2, shuffle = True, random_state = 8)
#Split remaining train into train and  validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2


print("X_train shape: {} \n".format(X_train.shape))
print("X_test shape: {} \n" .format(X_test.shape))
print("X_val shape: {} \n".format(X_val.shape))
print("y_train shape: {}\n".format(y_train.shape))
print("y_test shape: {}\n".format(y_test.shape))
print("y_val shape: {}\n".format(y_val.shape))


# Scale the featrues, you are fitting and transforming the train features but not the test/val. Testing data must remain unseen by the model. If you fit the testing data then your model learned on the testing data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

X_train shape: (26923, 5) 

X_test shape: (8975, 5) 

X_val shape: (8975, 5) 

y_train shape: (26923,)

y_test shape: (8975,)

y_val shape: (8975,)



In [13]:
%%timeit -r 1 -n 1

#Default Knn application and results
knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_val)
print(classification_report(y_val,y_pred))

print()

C:\Users\xrist\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


              precision    recall  f1-score   support

           0       0.77      0.85      0.81      4213
           1       0.86      0.77      0.81      4762

    accuracy                           0.81      8975
   macro avg       0.81      0.81      0.81      8975
weighted avg       0.81      0.81      0.81      8975


606 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
#Time how long it takes

%%timeit -r 1 -n 1


#GridSearch Knn with RepeatedStratifiedKFold application and results
knn2 = KNeighborsClassifier()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

n_neighbors = list(range(5,101,5))
weights = ['uniform', 'distance']
metric = ['minkowski', 'euclidean', 'manhattan']
param_grid = dict(n_neighbors=n_neighbors, weights=weights, metric=metric )

gridCV = GridSearchCV(knn2, param_grid, scoring='accuracy', n_jobs=-1, cv=cv, error_score=0, verbose=1)

grid_fit=gridCV.fit(X_train, y_train)

Fitting 30 folds for each of 120 candidates, totalling 3600 fits
Best: 0.829304 using {'metric': 'manhattan', 'n_neighbors': 50, 'weights': 'distance'}
Classification report 

               precision    recall  f1-score   support

           0       0.79      0.88      0.83      4213
           1       0.88      0.80      0.84      4762

    accuracy                           0.84      8975
   macro avg       0.84      0.84      0.84      8975
weighted avg       0.84      0.84      0.84      8975
 

Accuracy score '
': 0.8363231197771588
2min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [26]:
#GridSearch Knn with RepeatedStratifiedKFold application and results
knn2 = KNeighborsClassifier()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

n_neighbors = list(range(5,101,5))
weights = ['uniform', 'distance']
metric = ['minkowski', 'euclidean', 'manhattan']
param_grid = dict(n_neighbors=n_neighbors, weights=weights, metric=metric )

gridCV = GridSearchCV(knn2, param_grid, scoring='accuracy', n_jobs=-1, cv=cv, error_score=0, verbose=1)

grid_fit=gridCV.fit(X_train, y_train)

#Best parameters and accuracy
print("Best: %f using %s" % (grid_fit.best_score_, grid_fit.best_params_))

#Prediction on validation set with GridSearchCV
y_pred = grid_fit.predict(X_val)
print("Classification report", '\n'*2,classification_report(y_val,y_pred), '\n')
print("Accuracy score :" ,accuracy_score(y_val,y_pred), '\n')

Fitting 30 folds for each of 120 candidates, totalling 3600 fits
Best: 0.829304 using {'metric': 'manhattan', 'n_neighbors': 50, 'weights': 'distance'}
Classification report 

               precision    recall  f1-score   support

           0       0.79      0.88      0.83      4213
           1       0.88      0.80      0.84      4762

    accuracy                           0.84      8975
   macro avg       0.84      0.84      0.84      8975
weighted avg       0.84      0.84      0.84      8975
 

Accuracy score : 0.8363231197771588 



In [27]:
#Prediction on test set with GridSearchCV
y_pred = grid_fit.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))




              precision    recall  f1-score   support

           0       0.78      0.88      0.83      4275
           1       0.88      0.78      0.83      4700

    accuracy                           0.83      8975
   macro avg       0.83      0.83      0.83      8975
weighted avg       0.83      0.83      0.83      8975

0.8280779944289693


In [28]:
pl = pd.read_csv('political news for cv')
gp = pd.read_csv('gossip news for cv')

X_gp = gp.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]
y_gp = gp.label

X_pl = pl.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]
y_pl = pl.label


X_gp = sc.transform(X_gp)
X_pl = sc.transform(X_pl)

y_pred_pl = grid_fit.predict(X_pl)
y_pred_gp = grid_fit.predict(X_gp)

In [29]:
print('Report for the gossip set:', classification_report(y_pred_gp, y_gp))
print('Report for the political set:', classification_report(y_pred_pl, y_pl))

Report for the gossip set:               precision    recall  f1-score   support

           0       0.66      0.79      0.72     13911
           1       0.42      0.28      0.34      7711

    accuracy                           0.61     21622
   macro avg       0.54      0.53      0.53     21622
weighted avg       0.58      0.61      0.58     21622

Report for the political set:               precision    recall  f1-score   support

           0       0.76      0.69      0.72       490
           1       0.58      0.66      0.62       320

    accuracy                           0.68       810
   macro avg       0.67      0.68      0.67       810
weighted avg       0.69      0.68      0.68       810

